In [1]:
# Load Packages
import sys

sys.path.append('../../')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve
import pandas as pd
import re
import xgboost as xgb
import os
import pickle
import re
import csv
import codecs
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from joblib import dump
from string import punctuation
from importlib import reload
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from sklearn.metrics import fbeta_score, make_scorer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from internal.dataset_generator import page_info
import sys
csv.field_size_limit(sys.maxsize)
#reload(sys)
#sys.setdefaultencoding('utf-8')


131072

In [152]:
def get_matrix(matrix1, matrix2):
    matrix = np.zeros(matrix1.shape)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix1[i, j] == 0 and matrix2[i, j] == 0:
                matrix[i, j] = -1
            else:
                matrix[i, j] = (matrix1[i, j] * matrix2[i, j]) / (matrix1[i, j] + matrix2[i, j])
    return matrix

In [72]:
#6300

In [184]:
# Read Data
train = pd.read_csv('../dataset_generator/output/train.csv')
test = pd.read_csv('../dataset_generator/output/test.csv')

In [185]:
val = train[1000:]
train = train[:1000]

In [155]:
len(test)

676

In [186]:
# define tfidf vectorizer 
tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

In [187]:
BagOfWords = pd.concat([train.iloc[:, 1], train.iloc[:, 3]], axis = 0)

In [188]:
tfidf.fit(BagOfWords)

TfidfVectorizer(max_features=300, norm='l1', stop_words='english')

In [211]:
pickle.dump(tfidf, open('tfidf.pkl', "wb"))

In [213]:
vectorizer = pickle.load(open("tfidf.pkl",'rb'))

In [214]:
print(tfidf.vocabulary_)

{'french': 141, 'france': 140, 'left': 186, 'school': 251, 'age': 62, '14': 16, 'ref': 242, 'http': 160, 'com': 100, 'html': 159, 'url': 282, 'https': 161, 'web': 287, 'archive': 69, 'org': 216, 'date': 108, '2012': 36, '02': 3, '24': 50, 'april': 68, '2013': 37, 'style': 268, 'south': 258, 'en': 123, 'jpg': 172, 'category': 88, 'century': 92, 'country': 103, 'western': 290, 'europe': 129, 'la': 177, 'republic': 245, 'pp': 233, 'small': 257, 'title': 277, 'use': 283, 'british': 86, 'english': 126, 'february': 131, '2022': 46, 'march': 197, 'june': 174, 'nbsp': 206, 'flag': 136, 'svg': 270, 'national': 204, 'cite': 93, 'article': 72, 'ii': 163, 'used': 284, 'president': 234, 'www': 296, 'website': 288, 'language': 179, '20': 23, 'november': 211, 'lang': 178, 'br': 85, 'file': 132, '15': 17, 'european': 130, 'world': 295, 'red': 241, 'region': 243, 'union': 279, 'list': 188, 'location': 191, '21': 47, 'city': 95, 'official': 214, '2018': 42, 'people': 224, 'status': 267, 'dead': 111, '20

In [190]:
train_q1_tfidf = tfidf.transform(train.iloc[:, 1])
train_q2_tfidf = tfidf.transform(train.iloc[:, 3])

val_q1_tfidf = tfidf.transform(val.iloc[:, 1])
val_q2_tfidf = tfidf.transform(val.iloc[:, 3])

test_q1_tfidf = tfidf.transform(test.iloc[:, 1])
test_q2_tfidf = tfidf.transform(test.iloc[:, 3])

In [163]:
X = get_matrix(train_q1_tfidf, train_q2_tfidf)
y = train.iloc[:, 4]

X_test = get_matrix(test_q1_tfidf, test_q2_tfidf)
y_test = test.iloc[:, 4]

In [141]:
(X_test > 0.014).sum()

823

In [200]:
X = abs(train_q1_tfidf - train_q2_tfidf)
y = train.iloc[:, 4]

X_val = abs(val_q1_tfidf - val_q2_tfidf)
y_val = val.iloc[:, 4]

X_test = abs(test_q1_tfidf - test_q2_tfidf)
y_test = test.iloc[:, 4]

In [164]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [165]:
rf = RandomForestRegressor(n_estimators = 200,
                            min_samples_leaf = 10,
                            n_jobs = -1)

In [166]:
rf.fit(X_train, y_train)

RandomForestRegressor(min_samples_leaf=10, n_estimators=200, n_jobs=-1)

In [199]:
X_val.shape

(1000, 300)

In [167]:
pred_rf = rf.predict(X_test)

In [168]:
print(mean_squared_error(pred_rf, y_test))

0.605873187023968


In [171]:
# Training our model
params = {'learning_rate':0.01, 'n_estimators':600,
                    'nthread': 1, 'subsample': 0.6, 'min_child_weight': 1, 'max_depth': 3, 'gamma': 1.5, 'colsample_bytree': 0.8}
xgb_reg = XGBRegressor(**params)
#start_time = timer(None)
scores = cross_val_score(xgb_reg, X_train, y_train, scoring=make_scorer(mean_squared_error),  cv=4)
print(scores.mean(), scores)
#timer(start_time) 


0.45334764163608243 [0.44107139 0.51047786 0.46573497 0.39610635]


In [201]:
xgb_reg = XGBRegressor(**params)
xgb_reg.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=1.5, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=600, n_jobs=1, nthread=1,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.6,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [202]:
print(mean_squared_error(xgb_reg.predict(X), y))
print(mean_squared_error(xgb_reg.predict(X_test), y_test))
print(mean_squared_error(xgb_reg.predict(X_val), y_val))

0.19025644612363815
0.6671094463727923
0.5898964576106904


In [30]:
sorted(zip(rf.feature_importances_, list(tfidf.vocabulary_.keys())))

[(0.0, 'cricket'),
 (0.0, 'led'),
 (0.0, 'middle'),
 (0.0, 'nuclear'),
 (0.0, 'press'),
 (0.0, 'rights'),
 (0.0, 'song'),
 (0.0, 'washington'),
 (3.745237321867882e-06, 'fc'),
 (4.416418893083739e-06, '200'),
 (5.27246338745745e-06, 'energy'),
 (6.0636903150616655e-06, 'acre'),
 (8.479891830804833e-06, 'news'),
 (8.843873030982886e-06, '22'),
 (1.0604027284387733e-05, 'environment'),
 (1.3299359403263683e-05, 'financial'),
 (2.157224245034967e-05, 'nl'),
 (2.174948139413458e-05, 'association'),
 (2.2903390618721092e-05, '2011'),
 (2.630586922165466e-05, 'near'),
 (2.6404682344374925e-05, 'last3'),
 (2.86661339449792e-05, '2005'),
 (3.436951600317986e-05, '2021'),
 (3.458021571234275e-05, 'policy'),
 (3.581715707748625e-05, '42'),
 (3.598180899155991e-05, 'known'),
 (3.619949443618234e-05, 'goal'),
 (3.7570652282163776e-05, 'sort'),
 (4.0129406694112186e-05, 'white'),
 (4.018384712356731e-05, 'village'),
 (4.0355751953784954e-05, 'documents'),
 (4.4545610890969166e-05, 'publishing'),
 (

In [149]:
list(tfidf.vocabulary_.keys())

['french',
 'france',
 'left',
 'school',
 'age',
 '14',
 'art',
 'ref',
 'http',
 'com',
 'html',
 'url',
 'https',
 'web',
 'archive',
 'org',
 'date',
 '2012',
 '02',
 '24',
 'april',
 '2013',
 'style',
 'life',
 'south',
 'en',
 'jpg',
 'category',
 'century',
 'country',
 'western',
 'europe',
 'la',
 'republic',
 'pp',
 'small',
 'display',
 'title',
 'use',
 'british',
 'english',
 'february',
 '2022',
 'march',
 'june',
 'nbsp',
 'flag',
 'national',
 'cite',
 'article',
 'ii',
 'used',
 'president',
 'www',
 'website',
 'language',
 '20',
 'november',
 'lang',
 'br',
 'file',
 '15',
 'european',
 'world',
 'red',
 'region',
 'union',
 'list',
 'location',
 '21',
 'city',
 'official',
 '2018',
 'people',
 'status',
 'dead',
 '2011',
 '10',
 '16',
 'data',
 'countries',
 'population',
 '2016',
 'white',
 'history',
 'state',
 'west',
 'kingdom',
 'empire',
 'august',
 'july',
 '22',
 'september',
 'economic',
 'january',
 'october',
 'area',
 'gov',
 'publisher',
 'access',
 '20

In [147]:
xgb_reg.predict(X_test)

array([3.7175279, 3.4261334, 2.547948 , 2.531496 , 3.7175279, 4.047628 ,
       2.7609937, 2.7134862, 2.7134862, 2.3879852, 1.9748502, 2.3846383,
       3.243856 , 3.2714987, 2.8968024, 3.243856 , 3.2714987, 2.7092075,
       1.5618055, 3.243856 , 2.2645023, 2.590571 , 2.7166657, 2.6127853,
       2.575915 , 2.7166657, 2.2399917, 2.1885643, 2.8154168, 4.2428746,
       3.3106375, 3.8835084, 2.4981985, 2.637746 , 2.3554065, 2.409641 ,
       2.0775225, 4.2428746, 2.637746 , 2.3554065, 2.3554065, 2.110934 ,
       3.0501049, 1.798869 , 2.156961 , 2.2852292, 3.0501049, 3.0501049,
       2.3696618, 3.8890162, 3.4281385, 2.8288915, 3.8295667, 3.959404 ,
       2.6669133, 4.2912626, 4.351254 , 3.6347432, 2.5821052, 2.6369317,
       2.7411902, 3.7390943, 4.2912626, 4.351254 , 2.6369317, 2.6369317,
       4.011946 , 3.0807164, 2.2641435, 1.5291628, 4.011946 , 3.0807164,
       2.146009 , 4.011946 , 2.1363246, 2.2635722, 2.0674522, 4.622598 ,
       3.7989705, 2.9597566, 4.2293596, 4.2245097, 

In [148]:
y_test

0      4
1      3
2      3
3      1
4      4
      ..
671    3
672    2
673    2
674    2
675    2
Name: 2, Length: 676, dtype: int64

In [165]:
print((train_q1_tfidf != 0).sum())

325853


In [ ]:
test_q1_tfidf

In [168]:
print((test_q1_tfidf == 0).sum())

89452


# load model & predict

In [234]:
def predict(title1, title2):
    model_path = '0001.model'
    tfidf_path = "tfidf.pkl"
    bst = xgb.Booster({'nthread': 4})  # init model
    bst.load_model('0001.model')
    tfidf = pickle.load(open("tfidf.pkl",'rb'))
    #preprocessing
    predict_q1_tfidf = tfidf.transform(page_info.get(title1))
    predict_q2_tfidf = tfidf.transform(page_info.get(title2))
    X_pred = xgb.DMatrix(abs(predict_q1_tfidf - predict_q2_tfidf))
    predictions = bst.predict(X_pred)
    return predictions

In [238]:
predict("Huffington", "George_Knowling")

<class 'str'>
<class 'str'>


array([3.84074], dtype=float32)